In [0]:
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn .naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support
import pickle

import json
import numpy as np
import pandas as pd
import os

In [0]:
columns = ['score_overall', 'nose_score', 'nose_x', 'nose_y', 'leftEye_score', 'leftEye_x', 'leftEye_y',
              'rightEye_score', 'rightEye_x', 'rightEye_y', 'leftEar_score', 'leftEar_x', 'leftEar_y',
              'rightEar_score', 'rightEar_x', 'rightEar_y', 'leftShoulder_score', 'leftShoulder_x', 'leftShoulder_y',
              'rightShoulder_score', 'rightShoulder_x', 'rightShoulder_y', 'leftElbow_score', 'leftElbow_x',
              'leftElbow_y', 'rightElbow_score', 'rightElbow_x', 'rightElbow_y', 'leftWrist_score', 'leftWrist_x',
              'leftWrist_y', 'rightWrist_score', 'rightWrist_x', 'rightWrist_y', 'leftHip_score', 'leftHip_x',
              'leftHip_y', 'rightHip_score', 'rightHip_x', 'rightHip_y', 'leftKnee_score', 'leftKnee_x', 'leftKnee_y',
              'rightKnee_score', 'rightKnee_x', 'rightKnee_y', 'leftAnkle_score', 'leftAnkle_x', 'leftAnkle_y',
              'rightAnkle_score', 'rightAnkle_x', 'rightAnkle_y']
info = json.loads(open('key_points1.json', 'r').read())  #Put the name of the .json here
csv = np.zeros((len(info), len(columns)))
for i in range(csv.shape[0]):
    one = []
    one.append(info[i]['score'])
    for object in info[i]['keypoints']:
        one.append(object['score'])
        one.append(object['position']['x'])
        one.append(object['position']['y'])
    csv[i] = np.array(one)
df1=pd.DataFrame(csv, columns=columns) #dataframe

info = json.loads(open('key_points2.json', 'r').read())
csv = np.zeros((len(info), len(columns)))
for i in range(csv.shape[0]):
    one = []
    one.append(info[i]['score'])
    for object in info[i]['keypoints']:
        one.append(object['score'])
        one.append(object['position']['x'])
        one.append(object['position']['y'])
    csv[i] = np.array(one)
df2=pd.DataFrame(csv, columns=columns) #dataframe

info = json.loads(open('key_points3.json', 'r').read())
csv = np.zeros((len(info), len(columns)))
for i in range(csv.shape[0]):
    one = []
    one.append(info[i]['score'])
    for object in info[i]['keypoints']:
        one.append(object['score'])
        one.append(object['position']['x'])
        one.append(object['position']['y'])
    csv[i] = np.array(one)
df3=pd.DataFrame(csv, columns=columns) #dataframe

df = pd.concat([df1,df2,df3])
df['Class'] = 'buy'
df.to_csv(r'key_points_buy.csv')

In [0]:
df1 = pd.read_csv('key_points_buy.csv')
df2 = pd.read_csv('key_points_communicate.csv')
df3 = pd.read_csv('key_points_fun.csv')
df4 = pd.read_csv('key_points_hope.csv')
df5 = pd.read_csv('key_points_mother.csv')
df6 = pd.read_csv('key_points_really.csv')
df = pd.concat([df1,df2,df3,df4,df5,df6])
df.to_csv(r'key_points.csv')

In [0]:
df = df.drop(columns="Unnamed: 0")

In [0]:
X = df.values[:,0:52]
Y = df.values[:,52]

In [0]:
def kFoldCrossValidation(X,Y):

    kfold = model_selection.KFold(n_splits=10, random_state=42)
#    model = LogisticRegression()   
#    model = GaussianNB()  
    model = svm.NuSVC(gamma='auto')   
#    model = KNeighborsClassifier(n_neighbors=10, algorithm='auto' )  
#    model = RandomForestClassifier()   
#    model = DecisionTreeClassifier()   
    
    results = model_selection.cross_val_score(model, X, Y, cv=kfold)
    print("Accuracy after KFold Cross Validation: %.3f%% " % (results.mean()*100.0)) 

In [126]:
kFoldCrossValidation(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Accuracy after KFold Cross Validation: 11.462% 


In [0]:
def finalModelTestTrain(X,Y):
    
    #After kFoldCrossValidation select the below model that gives maximum accuracy
    model = svm.NuSVC(gamma='auto')

    test_size = 0.33
    seed = 42
    
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
    
    # Fit the model on training set
    model.fit(X_train, Y_train)

    filename = 'finalModel.pkl'
    pickle.dump(model, open(filename, 'wb'))
     

    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred=model.predict(X_test)
    
    result = loaded_model.score(X_test, Y_test)
    print("Accuracy of Trained Model: %.3f%% " % (result*100.0))
    print("Precison, Recall, F Socre:")
    print(precision_recall_fscore_support(Y_test, y_pred, average='macro'))